In [6]:
import cdflib
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [ ]:
cdf_file = cdflib.CDF('HaloCMEs/Data/AL1_ASW91_L2_BLK_20250626_UNP_9999_999999_V02.cdf')
print("Variables in CDF:")
print(cdf_file.cdf_info())

FileNotFoundError: D:\ISRO Hackthon\HaloCMEs\HaloCMEs\Data\AL1_ASW91_L2_TH2_20250626_UNP_9999_999999_V02.cdf not found